In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from scipy.io import mmread
sys.path.append(os.path.abspath(os.path.join('..', 'utils')))
import geo_utils
from geo_utils import read_prefixes_from_metadata_txt

root_path = '/var/datasets/single_papers/2023_feargal/GSE199750'


In [ ]:
metadata_df = read_prefixes_from_metadata_txt(f'{root_path}/GSE199750_series_matrix.txt')
print(metadata_df.shape)
metadata_df.head()

In [4]:
input_dir = f'{root_path}/GSE199750_RAW'
count_matrix = pd.DataFrame()


for file in os.listdir(input_dir):
    if file.endswith(".featureCounts.txt"):
        file_path = os.path.join(input_dir, file)
        data = pd.read_csv(file_path, comment='#', sep='\t')
        gene_counts = data[['Geneid', data.columns[-1]]]  # Last column is the count column
        sample_name = os.path.splitext(file)[0]
        gene_counts.columns = ['Geneid', sample_name]
        if count_matrix.empty:
            count_matrix = gene_counts
        else:
            count_matrix = pd.merge(count_matrix, gene_counts, on='Geneid', how='outer')
count_matrix.set_index('Geneid', inplace=True)

        

(60671, 260)


,GSM5983813_21-01684.featureCounts,GSM5983814_21-01685.featureCounts,GSM5983815_21-01686.featureCounts,GSM5983816_21-01687.featureCounts,GSM5983817_21-01688.featureCounts,GSM5983818_21-01689.featureCounts,GSM5983819_21-01690.featureCounts,GSM5983820_21-01691.featureCounts,GSM5983821_21-01692.featureCounts,GSM5983822_21-01693.featureCounts,...,GSM6395435_22-00308.featureCounts,GSM6395436_22-00309.featureCounts,GSM6395437_22-00310.featureCounts,GSM6395438_22-00312.featureCounts,GSM6395439_22-00659.featureCounts,GSM6395440_22-00660.featureCounts,GSM6395441_22-00662.featureCounts,GSM6395442_22-00666.featureCounts,GSM6395443_22-00667.featureCounts,GSM6395444_22-00668.featureCounts
Geneid,,,,,,,,,,,,,,,,,,,,,
ENSG00000223972,86,113,101,71,48,13,10,15,39,22,...,43,9,37,85,3,14,15,6,29,126
ENSG00000227232,49,32,87,73,70,43,30,80,86,136,...,62,105,81,86,69,24,59,18,28,42
ENSG00000278267,8,3,11,2,4,6,6,3,5,7,...,3,12,10,13,4,3,14,9,11,3
ENSG00000243485,0,0,2,0,1,1,0,0,0,3,...,0,0,2,0,0,0,1,0,0,0
ENSG00000284332,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
count_matrix_T = count_matrix.T
print(count_matrix_T.shape)
count_matrix_T.head()

(260, 60671)


Geneid,ENSG00000223972,ENSG00000227232,ENSG00000278267,ENSG00000243485,ENSG00000284332,ENSG00000237613,ENSG00000268020,ENSG00000240361,ENSG00000186092,ENSG00000238009,...,ENSG00000273532,ENSG00000276351,ENSG00000275661,ENSG00000277856,ENSG00000275063,ENSG00000271254,ENSG00000275405,ENSG00000275987,ENSG00000277475,ENSG00000268674
GSM5983813_21-01684.featureCounts,86,49,8,0,0,0,0,0,0,27,...,0,0,0,0,9,80,33,0,0,0
GSM5983814_21-01685.featureCounts,113,32,3,0,0,1,0,0,0,18,...,0,0,0,1,11,35,31,0,0,0
GSM5983815_21-01686.featureCounts,101,87,11,2,0,1,0,0,0,21,...,0,0,0,0,38,54,31,0,0,0
GSM5983816_21-01687.featureCounts,71,73,2,0,0,0,0,0,0,19,...,0,0,0,2,6,159,38,0,0,0
GSM5983817_21-01688.featureCounts,48,70,4,1,0,1,0,0,0,22,...,0,0,0,0,10,125,54,0,0,0
